In [19]:
import json
import spacy

from spacy.training import offsets_to_biluo_tags
from spacy.lang.de import German
from pathlib import Path

def validate_alignment(file_path):
    nlp = German()
    file_path = Path(file_path)

    with file_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    total_files = len(data)
    misaligned_files = 0

    for entry in data:
        text = entry["text"]
        labels_raw = entry.get("labels", [])
        labels = [(l["start"], l["end"], l["label"]) for l in labels_raw]
        doc = nlp.make_doc(text)

        try:
            tags = offsets_to_biluo_tags(doc, labels)
        except Exception as e:
            print(f"❌ Fehler bei Datei '{entry.get('file', 'unknown')}': {e}")
            continue

        if "-" in tags:
            misaligned_files += 1
            print(f"\n⚠️ Nicht ausrichtbare Labels in Datei: {entry.get('file', 'unknown')}")
            for l in labels:
                s, e, t = l
                original = text[s:e].replace("\n", "\\n")
                print(f"  → {t}: '{original}' @ {s}-{e}")

    print("\n🔍 Ergebnis:")
    print(f"  Insgesamt überprüft: {total_files}")
    print(f"  Davon mit fehlerhaften Labels: {misaligned_files}")

# 🔧 Aufruf:
validate_alignment("golden_dataset_norm_cleaned.json")


⚠️ Nicht ausrichtbare Labels in Datei: 35.txt
  → VORNAME: 'Gerolf' @ 177-183
  → NACHNAME: 'Buchholz' @ 184-192
  → FIRMA: 'Wende Stadelmann GmbH' @ 214-235
  → STRASSE: 'Carlos-Weiß-Allee 1-3' @ 236-257
  → POSTLEITZAHL: 'D-91531' @ 258-265
  → WOHNORT: 'Hildburghausen' @ 266-280
  → TELEFONNUMMER: '+49+49(0)4549 268653 11' @ 286-309
  → LINK: 'www.hgo-gmbh.de' @ 310-325
  → VORNAME: 'Gerolf' @ 333-339
  → NACHNAME: 'Buchholz' @ 340-348
  → FIRMA: 'Amtsgericht Hildburghausen HRB 89781\nVorstand: Hanspeter Ruppert, Heinfried Fischer\nAufsichtsratsvorsitzende: Dr. Astrid Köhler' @ 349-476
  → LINK: 'hgo-gmbh.de/datenschutz' @ 1293-1316
  → LINK: 'hgo-gmbh.de/en/data-privacy-statement' @ 1379-1416

⚠️ Nicht ausrichtbare Labels in Datei: 42.txt
  → VORNAME: 'Nerger' @ 164-170
  → NACHNAME: 'Grauel409291' @ 171-183
  → ZÄHLERNUMMER: '55Zählernummer:456123-5007643' @ 184-213

⚠️ Nicht ausrichtbare Labels in Datei: 117.txt
  → VERTRAGSNUMMER: '402071749' @ 101-110
  → VERTRAGSNUMMER: '4020

In [11]:
import json
from pathlib import Path


def anonymize_file(input_path, output_path):
    # Lade die JSON-Datei
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    anonymized_entries = []

    for entry in data:
        text = entry["text"]
        # Ersetze Labels von hinten nach vorne – kein Offset nötig!
        labels = sorted(entry["labels"], key=lambda l: l["start"], reverse=True)

        for label in labels:
            start, end = label["start"], label["end"]
            placeholder = f"[{label['label']}]"
            text = text[:start] + placeholder + text[end:]

        anonymized_entries.append({
            "file": entry["file"],
            "anonymized_text": text
        })

    # Speichere das Ergebnis als neue JSON-Datei
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(anonymized_entries, f, ensure_ascii=False, indent=2)

    print(f"✅ Anonymisierte Datei gespeichert unter: {output_path.resolve()}")

# Beispiel: Nutze den tatsächlichen Dateinamen, z. B. "granu_train.json"
input_file = Path("./golden_dataset_norm_cleaned.json")  # 👉 Passe das an deine hochgeladene Datei an
output_file = Path("./anonymized_output_cleaned.json")

anonymize_file(input_file, output_file)

✅ Anonymisierte Datei gespeichert unter: /Users/timonmartens/Library/CloudStorage/OneDrive-Persönlich/Desktop/Veranstaltungen/Data Analytics in Applications/daia-eon/data/original/anonymized_output_cleaned.json
